In [ ]:
import os
import shutil
from bmtk.builder.networks import NetworkBuilder
from bmtk.utils.sim_setup import build_env_bionet
from bmtk.simulator import bionet

import h5py
import matplotlib.pyplot as plt
import numpy as np
from workshop2025_activityutils import calc_rate_loss

## Goal: Create a biophysical neuron model with 3 virtual input populations, that replicates the provided target output trace.

Provided:
* cell morphology
* cell dynamic param file
* synapse templates
* synapse dynamic param files
* Input population spike files

Fill in the blanks for each section to simulate a biophysical neuron targetted by 3 distinct input populations. Try to match the target output as closely as possible. If there is a mismatch test out different combinations of parameter choices.

Use tutorial 2 as a reference. 

### Step 1: Build a biophysical neuron

In [ ]:
base_dir = './activity_files'

test_file_dir = os.path.join(base_dir, 'test_files') # dir to provided files
net_dir = os.path.join(test_file_dir, 'network')
input_dir = os.path.join(base_dir, 'inputs')
test_path = os.path.join(test_file_dir, 'output')
target_path = os.path.join(base_dir, 'targets')

if os.path.exists(net_dir):
    shutil.rmtree(net_dir)

# Build the biophysical neuron
# fill in the blanks here, look at tutorial 2 if you need help
bneuron = NetworkBuilder('bneuron')
bneuron.add_nodes(
    cell_name='biophysical_neuron',
    potential='exc',
    model_type='',
    model_template='',
    model_processing='',
    dynamics_params='473863035_fit.json', # dynamics parameters for the neuron, look at this file to see what parameters are used, do you know what each represents?
    morphology='Rorb_325404214_m.swc' # feel free to play around with different morphologies, do they significantly change the results?
)


bneuron.build()
bneuron.save_nodes(output_dir= net_dir)

### Step 2: Define the input populations

Choose the name of the first input population and create 2 more 'virtual' input populations.


In [ ]:
# Input 1
input1 = NetworkBuilder('input1')
input1.add_nodes(
    N=10, # number of nodes in the population
    pop_name='', # name of the population, this will be used to identify the population in the output files
    potential='exc', # potential of the input network, can be 'exc' or 'inh'
    model_type='virtual'
)

# Input 2

# Input 3

### Step 3: Connect the inputs to the main neuron

This is where you design the connection rule between your inputs and target neuron! What does each argument do? How do you choose what to put? 

If your simulation output looks different from the target play around with these fields, how does each one change the resulting output?

(Hint: you might also need to modify the potential of your input networks)

In [ ]:
# input 1
input1.add_edges(
    source={'pop_name': 'input1'}, target=bneuron.nodes(),
    connection_rule=10, # connection rule for the edges, for this exercise set as an integer (number of connections per neuron) 
    syn_weight=0.001, # synaptic weight for the edges, this is the strength of the connection
    delay= 2.0, # delay for the edges, provided.
    weight_function=None, # weight function for the edges, can be None or a string pointing to a weight function file. Set to None for this exercise.
    target_sections=[''], # section of the neuron to connect to, can be 'soma', 'apical', or 'basal'
    distance_range=[0.0, 150.0], # range of the cell section to connect to, provided for input 1 (not necessarily the same for all 3 inputs).
    dynamics_params='', # dynamics parameters for the synapse, look at the provided files to see what options are available.
    model_template='' # model template for the synapse, look at the provided test_files/synaptic_models dir to see what options are available.
)

input1.build()
input1.save_nodes(output_dir=net_dir)
input1.save_edges(output_dir=net_dir)

# input 2

# input 3


### Step 4: Build the network and run the simulation

If this cell throws a hocobj_call error try restarting the kernel and running again. 

In [ ]:
dt = 0.01  # time step for the simulation
build_env_bionet(
    base_dir=test_file_dir,
    config_file='config.json',
    network_dir=net_dir,
    tstop=3000.0, dt=dt,
    report_vars=['v', 'cai'],    # Record membrane potential and calcium (default recording from the soma)
    spikes_inputs=[('input1', 
                    os.path.join(input_dir, 'input1_spikes.h5')),
                    ('input2', 
                    os.path.join(input_dir,'input2_spikes.h5')),
                    ('input3', 
                    os.path.join(input_dir,'input3_spikes.h5'))],
    include_examples=True,       # Copies components files, only run once set to false after
    compile_mechanisms=True,      # Will try to compile NEURON mechanisms, only run once set to false after
    overwrite_config=True, # Overwrite the config file if it exists
)

conf = bionet.Config.from_json(os.path.join(test_file_dir,'config.json'))
conf.build_env()
net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

### Step 5: Examine results and compare to target traces

In [ ]:
# load in the simulation output files
with h5py.File(os.path.join(test_path, 'v_report.h5'), 'r') as file:
    test_vData = file['report']['bneuron']['data'][:]

with h5py.File(os.path.join(test_path, 'cai_report.h5'), 'r') as file:
    test_cData = file['report']['bneuron']['data'][:]

with h5py.File(os.path.join(test_path, 'spikes.h5'), 'r') as file:
    print(file['spikes']['bneuron']['timestamps'].shape)
    test_spikes = file['spikes']['bneuron']['timestamps'][:]

# load in target data
with h5py.File(os.path.join(target_path, 'v_report.h5'), 'r') as file:
    target_vData = file['report']['bneuron']['data'][:]

with h5py.File(os.path.join(target_path, 'cai_report.h5'), 'r') as file:
    target_cData = file['report']['bneuron']['data'][:]

with h5py.File(os.path.join(target_path, 'spikes.h5'), 'r') as file:
    target_spikes = file['spikes']['bneuron']['timestamps'][:]

# calculate the rate loss
rate_loss, target_rate, test_rate = calc_rate_loss(test_vData, target_vData, -10, dt, 1000)

# plot target and test data
time = np.linspace(0, len(target_vData) * 0.1, len(target_vData))
fig, ax = plt.subplots(3, 1, figsize=(10, 6))
ax[0].plot(time, target_vData)
ax[0].plot(time, test_vData, alpha = 0.7)
ax[0].set_title('Membrane Potential')
ax[0].set_xlabel('Time (ms)')
ax[0].set_xlim([min(time), max(time)])

ax[1].plot(time, target_rate)
ax[1].plot(time, test_rate, alpha = 0.7)
ax[1].set_title('Smooth Firing Rate')
ax[1].set_xlabel('Time (ms)')
ax[1].set_xlim([min(time), max(time)])

ax[2].plot(time, target_cData, label='Target')
ax[2].plot(time, test_cData, label='Test', alpha = 0.7)
ax[2].legend()
ax[2].set_title('Calcium Concentration')
ax[2].set_xlabel('Time (ms)')
ax[2].set_xlim([min(time), max(time)])

plt.tight_layout()

print('Number of Target Spikes: {}'.format(len(target_spikes)))
print('Number of Test Spikes: {}'.format(len(test_spikes)))
print(f'Rate loss: {rate_loss}')
